In [2]:
import duckdb

In [3]:
conn = duckdb.connect()

In [30]:
duckdb.sql(
    """SELECT *
           FROM read_json_auto('Data/*/*.json') LIMIT 1"""
           )



┌─────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [35]:
duckdb.sql(
    """SELECT url as game_url,
                time_control as time_control,
                rated as rated,
                time_class as time_class,
                rules as rules,
                white.rating as white_rating,
                white.result as white_result,
                black.rating as black_rating,
                black.result as black_result,    
                REGEXP_EXTRACT(pgn, '\[Event "(.*?)"', 1) as pgn_event,
                REGEXP_EXTRACT(pgn, '\[Site "(.*?)"', 1) as pgn_site,
                REGEXP_EXTRACT(pgn, '\[Date "(.*?)"', 1) as pgn_date, 
                REGEXP_EXTRACT(pgn, '\[White "(.*?)"', 1) as pgn_white_user,   
                REGEXP_EXTRACT(pgn, '\[Black "(.*?)"', 1) as pgn_black_user,
                REGEXP_EXTRACT(pgn, '\[Result "(.*?)"', 1) as pgn_result,
                REGEXP_EXTRACT(pgn, '\[CurrentPosition "(.*?)"', 1) as pgn_current_position,
                REGEXP_EXTRACT(pgn, '\[Timezone "(.*?)"', 1) as pgn_timezone,
                REGEXP_EXTRACT(pgn, '\[ECO "(.*?)"', 1) as pgn_eco,
                REGEXP_EXTRACT(pgn, '\[ECOUrl "(.*?)"', 1) as pgn_eco_url,
                REGEXP_EXTRACT(pgn, '\[StartTime "(.*?)"', 1) as start_time,
                REGEXP_EXTRACT(pgn, '\[EndTime "(.*?)"', 1) as End_time,
                ARRAY_TO_STRING(REGEXP_EXTRACT_ALL(pgn, '\. (.*?) {\[', 1), ' ') as pgn_test
           FROM read_json_auto('Data/*/*.json') LIMIT 10"""
           )



┌─────────────────────────────────────────────┬──────────────┬─────────┬────────────┬─────────┬──────────────┬──────────────┬──────────────┬──────────────┬────────────┬───────────┬────────────┬────────────────┬────────────────┬────────────┬───────────────────────────────────────────────────────────────┬──────────────┬─────────┬───────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬──────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                  game_url                   │ time_control │  rated  │ time_class │  rules  │ white_rating │ white_r

In [ ]:
# Create Dimension tables for Openings, Date, Player and GameType

# Openings Dimension Table
conn.execute("""
CREATE TABLE IF NOT EXISTS Openings AS
SELECT DISTINCT
    pgn_eco AS opening_code,
    pgn_eco_url AS opening_url
FROM read_json_auto('Data/*/*.json')
""")

# Date Dimension Table
conn.execute("""
CREATE TABLE IF NOT EXISTS Date AS
SELECT DISTINCT
    pgn_date AS game_date,
    start_time,
    End_time
FROM read_json_auto('Data/*/*.json')
""")

# Player Dimension Table
conn.execute("""

# Openings Dimension Table
conn.execute("""
CREATE TABLE IF NOT EXISTS Openings AS
SELECT DISTINCT
    pgn_eco AS opening_code,
    pgn_eco_url AS opening_url
FROM read_json_auto('Data/*/*.json')
""")

# Date Dimension Table
conn.execute("""
CREATE TABLE IF NOT EXISTS Date AS
SELECT DISTINCT
    pgn_date AS game_date,
    start_time,
    End_time
FROM read_json_auto('Data/*/*.json')
""")

# Player Dimension Table
conn.execute("""


In [10]:
# Transform openings csv 
openings = conn.sql("""
            SELECT "eco-volume" as eco_family, eco, name, pgn
            FROM 'hf://datasets/Lichess/chess-openings/data/train-00000-of-00001.parquet'
         """
).df()
openings


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,eco_family,eco,name,pgn
0,A,A00,Amar Opening,1. Nh3
1,A,A00,Amar Opening: Paris Gambit,1. Nh3 d5 2. g3 e5 3. f4
2,A,A00,"Amar Opening: Paris Gambit, Gent Gambit",1. Nh3 d5 2. g3 e5 3. f4 Bxh3 4. Bxh3 exf4 5. ...
3,A,A00,Amsterdam Attack,1. e3 e5 2. c4 d6 3. Nc3 Nc6 4. b3 Nf6
4,A,A00,Anderssen's Opening,1. a3
...,...,...,...,...
3508,E,E98,"King's Indian Defense: Orthodox Variation, Cla...",1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...
3509,E,E98,"King's Indian Defense: Orthodox Variation, Cla...",1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...
3510,E,E98,"King's Indian Defense: Orthodox Variation, Cla...",1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...
3511,E,E99,"King's Indian Defense: Orthodox Variation, Cla...",1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...


In [16]:
conn.sql("""SELECT *  
            FROM openings
            WHERE openings.eco = "A06"
         """)

BinderException: Binder Error: Referenced column "A06" not found in FROM clause!
Candidate bindings: "openings.eco"